In [1]:
import dask.distributed as dd
from dask.distributed import Client, LocalCluster, progress
from dask_jobqueue import PBSCluster
from distributed.utils import tmpfile
from dask.distributed import get_worker
import os

/jobfs/93006228.gadi-pbs/ipykernel_692322/1276645085.py:4: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [4]:
# The jupyter notebook is launched from your $HOME directory.
# Change the working directory to the C-Debugging directory
# which was created in your username directory under /scratch/vp91
os.chdir(os.path.expandvars("/scratch/vp91/AAPP2023/AAPP-Distributed-Dask/Dask-distributed"))

In [6]:
os.environ['DASK_PYTHON'] = '/scratch/vp91/AAPP2023/dask-python-venv/bin/python3'
setup_commands = ["module load python3/3.9.2", "source /scratch/vp91/AAPP2023/dask-python-venv/bin/activate"]
extra = ['-q normal',
         '-P vp91', 
         '-l ncpus=48', 
         '-l mem=192GB']

cluster = PBSCluster(walltime="00:50:00", 
                     cores=48, 
                     memory="192GB",
                     #processes=48, 
                     shebang='#!/usr/bin/env bash',
                     job_extra_directives=extra, 
                     local_directory='$TMPDIR', 
                     job_directives_skip=["select"], 
                     interface="ib0",
                     job_script_prologue=setup_commands,
                     python=os.environ["DASK_PYTHON"])

/scratch/vp91/AAPP2023/dask-python-venv/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39461 instead
  warnings.warn(


In [7]:
print(cluster.job_script())

#!/usr/bin/env bash

#PBS -N dask-worker
#PBS -l walltime=00:50:00
#PBS -q normal
#PBS -P vp91
#PBS -l ncpus=48
#PBS -l mem=192GB
module load python3/3.9.2
source /scratch/vp91/AAPP2023/dask-python-venv/bin/activate
/scratch/vp91/AAPP2023/dask-python-venv/bin/python3 -m distributed.cli.dask_worker tcp://10.6.28.24:43639 --nthreads 6 --nworkers 8 --memory-limit 22.35GiB --name dummy-name --nanny --death-timeout 60 --local-directory $TMPDIR --interface ib0



In [8]:
cluster.scale(jobs=2)

In [15]:
cluster

PBSCluster(f3ceab2d, 'tcp://10.6.28.24:43639', workers=16, threads=96, memory=357.60 GiB)

In [16]:
client = Client(cluster)

In [17]:
client

<Client: 'tcp://10.6.28.24:43639' processes=16 threads=96, memory=357.60 GiB>

In [18]:
cluster.scheduler

<Scheduler 'tcp://10.6.28.24:43639', workers: 16, cores: 96, tasks: 0>

cluster.get_logs()

In [19]:
!which python

/scratch/vp91/AAPP2023/dask-python-venv/bin/python


In [14]:
!qstat

Job id                 Name             User              Time Use S Queue
---------------------  ---------------- ----------------  -------- - -----
93005055.gadi-pbs      sys-dashboard-s* fxf900            00:03:40 R gpuvolta-exec   
93006228.gadi-pbs      sys-dashboard-s* fxf900            00:00:29 R gpuvolta-exec   
93006746.gadi-pbs      dask-worker      fxf900            00:00:00 R normal-exec     
93006747.gadi-pbs      dask-worker      fxf900            00:00:00 R normal-exec     


In [39]:
!pwd

/scratch/vp91/jj8451


In [20]:
def slow_increment(x):
    return x+1

In [21]:
futures = client.submit(slow_increment, 5000)

In [22]:
futures

<Future: finished, type: int, key: slow_increment-b81aef4bc1e6be1bd546aaa379a07482>

In [23]:
progress(futures)

VBox()

/scratch/vp91/AAPP2023/dask-python-venv/lib/python3.9/site-packages/distributed/scheduler.py:5711: UserWarning: Scheduler already contains a plugin with name websocket; overwriting.
  warnings.warn(
